In [3]:
import pyigtl
from matplotlib import pyplot as plt
import numpy as np
from time import sleep
from YOLOv5 import YOLOv5
from IPython.display import clear_output
import torch


weights = f'D:/GitRepos/aigt-LIVE/UltrasoundObjectDetection/YOLOv5/dummyRun/weights/best.pt'
target_size = 512
line_thickness = 2
model = None
host = "127.0.0.1"
port = 18944
#model = YOLOv5(weights=weights, device=device)

client = pyigtl.OpenIGTLinkClient(host=host, port=port)

while True:
    message = client.wait_for_message("Image_Reference", timeout=3)
    if message:
        if model is None:
            input_size = message.image.shape[1:3]
            device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
            model = YOLOv5(weights=weights,
                           device=device,
                           line_thickness=line_thickness,
                           input_size=input_size,
                           target_size=target_size)

        image = np.rot90(np.transpose(message.image, (1,2,0)), 2)

        prediction = model.predict(image)
        image_message = pyigtl.ImageMessage(np.flip(np.flip(prediction, axis=1), axis=2), device_name="TransformedImage5")
        client.send_message(image_message, wait=True)
        



Fusing layers... 
Model summary: 157 layers, 7031701 parameters, 0 gradients, 15.8 GFLOPs


KeyboardInterrupt: 